In [1]:
import pandas as pd
import json

In [11]:
df = pd.read_csv('../semEval16/check_correctness/graph-train-correctness.csv', encoding='utf-8')
df

,sent_id,text,Source,Target,Polar_expression,Polarity,Target_matching
0,1004293:0,Judging from previous posts this used to be a ...,NaN,NaN,not any longer.,Negative,False
1,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,-,Positive,False
2,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,there were,Positive,False
3,1004293:2,"They never brought us complimentary noodles, i...",NaN,NaN,NaN,NaN,False
4,1004293:3,The food was lousy - too sweet or too salty an...,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
2552,FF#10:7,"I can't believe that it was, but please put th...",NaN,NaN,NaN,NaN,False
2553,FF#10:8,The waitress came to check in on us every few ...,NaN,NaN,NaN,NaN,False
2554,FF#10:9,I couldn't ignore the fact that she reach over...,NaN,NaN,NaN,NaN,False
2555,FF#10:10,She then put the check down without asking if ...,NaN,NaN,NaN,NaN,False


In [10]:
# open training data or test data
with open('../semEval16/graph_parser/predictions/train/opener_en_prediction.json') as f:
    train_json = json.load(f)

train_json[100:115]

[{'sent_id': '1123262:0',
  'text': "I think I've had some the best meals of my life at minnow.",
  'opinions': [{'Source': [[], []],
    'Target': [['meals'], ['31:36']],
    'Polar_expression': [['best'], ['26:30']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}]},
 {'sent_id': '1123262:1',
  'text': "The seafood is amazing, there's a good wine list, and the ever-changing menu always offers some great surprises.",
  'opinions': [{'Source': [[], []],
    'Target': [['the ever-changing menu'], ['54:76']],
    'Polar_expression': [['great surprises.'], ['96:112']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'},
   {'Source': [[], []],
    'Target': [['the ever-changing menu'], ['54:76']],
    'Polar_expression': [['always offers'], ['77:90']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'},
   {'Source': [[], []],
    'Target': [[], []],
    'Polar_expression': [['list,'], ['44:49']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}]},
 {'sent_id'

In [19]:
for i in train_json:
    id = i['sent_id']

    for j in i['opinions']:
        # j['Matching'] = True
        # print(j)
        for k in range(len(df)):
            if (id == df.iloc[k]['sent_id']) and (pd.isnull(df.iloc[k]['Target']) == False):
                if (df.iloc[k]['Target_matching'] == True) and (df.iloc[k]['Target'] == )
                    j['Matching'] = True
                else:
                    j['Matching'] = False
            else:
                j['Matching'] = False

{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [['not any longer.'], ['62:77']], 'Polarity': 'Negative', 'Intensity': 'Standard', 'Matching': True}
{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [['-'], ['65:66']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': True}
{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [['there were'], ['4:14']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': True}
{'Source': [[], []], 'Target': [['the food'], ['34:42']], 'Polar_expression': [['always consistently, outrageously good.'], ['46:85']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': True}
{'Source': [[], []], 'Target': [['restaurant'], ['17:27']], 'Polar_expression': [['the best'], ['8:16']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': True}
{'Source': [[], []], 'Target': [['The duck confit'], ['0:15']], 'Polar_expression': [['always amazing'], ['19:33']], 'Polarity': 'Positive', 'Intensity': 'Sta